# Experimental features

Experimental features for modelbase2.

All APIs shown should be considered unstable and may change without notice.

In [ ]:
from example_models import get_linear_chain_1v, get_linear_chain_2v
from modelbase2.experimental import (
    generate_model_code_py,
    generate_modelbase_code,
    model_diff,
    to_tex,
)

## code generation

> Currently, the limitation here is that functions used for reactions etc. cannot call other functions.  

`modelbase2` can generate own source code from a model.  

In [ ]:
print(generate_modelbase_code(get_linear_chain_1v()))

`modelbase2` can also generate a generic python function from the source code.  
The plan here is to generalise this to be able to export models into other programming languages as well.  

In [ ]:
print(generate_model_code_py(get_linear_chain_2v()))

## Diffs

`modelbase2` can generate diffs between two models to quickly analyse differences between all model elements.  

In [ ]:
print(
    model_diff(
        get_linear_chain_2v(),
        get_linear_chain_1v(),
    )
)

In [ ]:
print(
    model_diff(
        get_linear_chain_2v(),
        get_linear_chain_2v().scale_parameter("k3", 2.0),
    )
)

## LaTeX export

> Currently, the limitation here is that functions used for reactions etc. cannot call other functions.  

`modelbase2` supports writing out your model as `LaTeX`.  

In [ ]:
print(to_tex(get_linear_chain_1v()))

## Symbolic models & identifiability analysis

In [ ]:
import sympy

from modelbase2.experimental import strikepy
from modelbase2.experimental.symbolic import SymbolicModel, to_symbolic_model
from modelbase2.model import Model


def check_identifiability(
    sym_model: SymbolicModel, outputs: list[sympy.Symbol]
) -> strikepy.Result:
    strike_model = strikepy.Model(
        states=list(sym_model.variables.values()),
        pars=list(sym_model.parameters.values()),
        eqs=sym_model.eqs,
        outputs=outputs,
    )
    return strikepy.strike_goldd(strike_model)


def infect(s: float, i: float, n: float, beta: float) -> float:
    return beta / n * i * s


def recover(i: float, gamma: float) -> float:
    return gamma * i


def total_population(s: float, i: float, r: float) -> float:
    return s + i + r


def sir() -> Model:
    return (
        Model()
        .add_parameters({"beta": 1.0, "gamma": 0.1})
        .add_variables({"s": 99, "i": 1, "r": 0})
        .add_derived("n", total_population, args=["s", "i", "r"])
        .add_reaction(
            "infect",
            infect,
            args=["s", "i", "n", "beta"],
            stoichiometry={"s": -1, "i": 1},
        )
        .add_reaction(
            "recover", recover, args=["i", "gamma"], stoichiometry={"i": -1, "r": 1}
        )
    )


model = sir()
sym_model = to_symbolic_model(model)
res = check_identifiability(sym_model, [sympy.Symbol("i"), sympy.Symbol("r")])
print(res.summary())